In [151]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
cat = CatBoostRegressor()

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [152]:
train = pd.read_csv('G:/내 드라이브/Github/BigContest_Flood-ZERO/고경수/train.csv')
test = pd.read_csv('G:/내 드라이브/Github/BigContest_Flood-ZERO/고경수/test.csv')
submission = pd.read_csv('G:/내 드라이브/Github/BigContest_Flood-ZERO/고경수/submission.csv')

In [153]:
유역평균강수 = ['유역평균강수1','유역평균강수2','유역평균강수3','유역평균강수4','유역평균강수5','유역평균강수6']
강우A = ['강우A1','강우A2','강우A3','강우A4','강우A5','강우A6']
강우B = ['강우B1','강우B2','강우B3','강우B4','강우B5','강우B6']
강우C = ['강우C1','강우C2','강우C3','강우C4','강우C5','강우C6']
강우D = ['강우D1','강우D2','강우D3','강우D4','강우D5','강우D6']
수위E = ['수위E1','수위E2','수위E3','수위E4','수위E5','수위E6']
수위D = ['수위D1','수위D2','수위D3','수위D4','수위D5','수위D6']

---
# 사상번호별 인덱스 정규화 추가

In [154]:
X = train.drop('유입량', axis=1)
y = train['유입량']

In [155]:
X_1 = X.iloc[:226]
X_2 = X.iloc[226:326]
X_3 = X.iloc[326:407]
X_4 = X.iloc[407:441]
X_5 = X.iloc[441:535]
X_6 = X.iloc[535:581]
X_7 = X.iloc[581:632]
X_8 = X.iloc[632:673]
X_9 = X.iloc[673:741]
X_10 = X.iloc[741:827]
X_11 = X.iloc[827:937]
X_12 = X.iloc[937:1064]
X_13 = X.iloc[1064:1131]
X_14 = X.iloc[1131:1181]
X_15 = X.iloc[1181:1443]
X_16 = X.iloc[1443:1648]
X_17 = X.iloc[1648:1773]
X_18 = X.iloc[1773:1838]
X_19 = X.iloc[1838:1933]
X_20 = X.iloc[1933:2197]
X_21 = X.iloc[2197:2281]
X_22 = X.iloc[2281:2565]
X_23 = X.iloc[2565:2694]
X_24 = X.iloc[2694:2788]
X_25 = X.iloc[2788:2891]

In [156]:
dtlists = []
index = pd.DataFrame()
for j in range(1, 26):
    eval('dtlists.append(X_'+str(j)+')')
    dtlists[j-1] = dtlists[j-1].reset_index()
    dtlists[j-1]['인덱스'] = dtlists[j-1].index
    dtlists[j-1]['정규화인덱스'] = dtlists[j-1]['인덱스'] / (len(dtlists[j-1])-1)
    index = pd.concat([index, pd.DataFrame(dtlists[j-1])])
    
index = index.reset_index()
train['정규화인덱스'] = index['정규화인덱스']

In [157]:
X_train, X_test, y_train, y_test = train_test_split(train[['정규화인덱스',*유역평균강수, *강우A, *강우B, *강우C, *강우D, *수위E, *수위D]],
                                                           train['유입량'], test_size=0.2109996, shuffle=False)

Original_train = X_train # 사상번호 1 ~ 21
Original_test = X_test # 사상번호 22 ~ 25

---
# 정규화 인덱스 추가 (집단5)

In [145]:
X_train = Original_train[['정규화인덱스',유역평균강수[4],강우A[4], 강우B[4], 강우C[4], 강우D[4],수위D[4], 수위E[4]]]
X_test = Original_test[['정규화인덱스',유역평균강수[4],강우A[4], 강우B[4], 강우C[4], 강우D[4],수위D[4], 수위E[4]]]

model = cat
model.fit(X_train, y_train)

Learning rate set to 0.045201
0:	learn: 2236.1668230	total: 10.9ms	remaining: 10.9s
1:	learn: 2160.0037963	total: 21.8ms	remaining: 10.9s
2:	learn: 2087.8194984	total: 30.8ms	remaining: 10.2s
3:	learn: 2017.4565989	total: 39.8ms	remaining: 9.9s
4:	learn: 1946.2035582	total: 49.4ms	remaining: 9.82s
5:	learn: 1880.2293331	total: 56.8ms	remaining: 9.41s
6:	learn: 1816.1895236	total: 63.7ms	remaining: 9.04s
7:	learn: 1758.3634566	total: 74.3ms	remaining: 9.21s
8:	learn: 1701.7040691	total: 82.3ms	remaining: 9.06s
9:	learn: 1646.7363090	total: 90.3ms	remaining: 8.94s
10:	learn: 1594.2289122	total: 98.9ms	remaining: 8.89s
11:	learn: 1543.2763398	total: 106ms	remaining: 8.75s
12:	learn: 1498.1519514	total: 118ms	remaining: 8.94s
13:	learn: 1451.6544753	total: 145ms	remaining: 10.2s
14:	learn: 1405.7064355	total: 151ms	remaining: 9.93s
15:	learn: 1362.9803350	total: 155ms	remaining: 9.56s
16:	learn: 1321.7854178	total: 161ms	remaining: 9.3s
17:	learn: 1281.5627142	total: 165ms	remaining: 9.02s

In [146]:
pred = model.predict(X_test)

In [147]:
rmse = (np.sqrt(np.mean(mean_squared_error(y_test, pred))))
rmse

401.55289474874183

In [158]:
from catboost import CatBoostRegressor, Pool
from catboost.utils import get_roc_curve

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2109996, shuffle=False)
X_train = Original_train[['정규화인덱스',유역평균강수[4],강우A[4], 강우B[4], 강우C[4], 강우D[4],수위D[4], 수위E[4]]]
X_test = Original_test[['정규화인덱스',유역평균강수[4],강우A[4], 강우B[4], 강우C[4], 강우D[4],수위D[4], 수위E[4]]]

model = CatBoostRegressor()
model.fit(X_train, y_train)
pool = Pool(X_train, y_train)

get_roc_curve(model, 
              pool,
              plot=True)

Learning rate set to 0.045201
0:	learn: 2236.1668230	total: 8.01ms	remaining: 8.01s
1:	learn: 2160.0037963	total: 13.3ms	remaining: 6.63s
2:	learn: 2087.8194984	total: 18.5ms	remaining: 6.16s
3:	learn: 2017.4565989	total: 25.4ms	remaining: 6.33s
4:	learn: 1946.2035582	total: 30.5ms	remaining: 6.07s
5:	learn: 1880.2293331	total: 37.1ms	remaining: 6.15s
6:	learn: 1816.1895236	total: 51.1ms	remaining: 7.24s
7:	learn: 1758.3634566	total: 56.6ms	remaining: 7.02s
8:	learn: 1701.7040691	total: 65.5ms	remaining: 7.21s
9:	learn: 1646.7363090	total: 70.9ms	remaining: 7.02s
10:	learn: 1594.2289122	total: 75.6ms	remaining: 6.8s
11:	learn: 1543.2763398	total: 81ms	remaining: 6.67s
12:	learn: 1498.1519514	total: 85.8ms	remaining: 6.51s
13:	learn: 1451.6544753	total: 92.4ms	remaining: 6.51s
14:	learn: 1405.7064355	total: 101ms	remaining: 6.66s
15:	learn: 1362.9803350	total: 108ms	remaining: 6.66s
16:	learn: 1321.7854178	total: 115ms	remaining: 6.64s
17:	learn: 1281.5627142	total: 126ms	remaining: 6.8

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/roc_curve.cpp:113: No documents of class 0.